In [1]:
import tensorflow as tf
import re
import numpy as np

In [2]:
# Data Took From WikiPedia = https://en.wikipedia.org/wiki/Tata_Consultancy_Services
with open("data", "rb") as f:
    raw_data = f.read().decode()

In [3]:
def clean_text(text):
    tokens = re.split("\W+", text)
    tokens = [w for w in tokens if w.isalpha()]
    tokens = [w.lower() for w in tokens]
    return tokens

In [4]:
extracted_tokens = clean_text(raw_data)
extracted_tokens[:10]

['tata',
 'consultancy',
 'services',
 'tcs',
 'is',
 'an',
 'indian',
 'multinational',
 'information',
 'technology']

In [5]:
print ("Original Length: ", len(extracted_tokens))
print ("Unique WordList: ", len(set(extracted_tokens)))

Original Length:  783
Unique WordList:  309


In [6]:
length = 51
lines = list()

for index in range(length, len(extracted_tokens)):
    sequence = extracted_tokens[index - length : index]
    line = " ".join(sequence)
    lines.append(line)

In [7]:
print(len(lines))

732


In [8]:
lines[0]

'tata consultancy services tcs is an indian multinational information technology it services and consulting company headquartered in mumbai maharashtra india with its largest campus located in chennai tamil nadu india as of february tcs is the largest it services company in the world by market capitalisation billion it is a subsidiary'

In [9]:
extracted_tokens[51]

'of'

## Building LSTM Networks

In [10]:
tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(lines)

In [11]:
sequences = np.array(tokenizer.texts_to_sequences(lines))
print(sequences.shape)

(732, 51)


In [12]:
X, y = sequences[:, :-1], sequences[:, -1]

In [13]:
VOCAB_SIZE = len(tokenizer.word_index) + 1
SEQUENCE_LEN = X.shape[1]

print (VOCAB_SIZE, SEQUENCE_LEN)

310 50


In [14]:
X[33], y[33]

(array([  3,  29,   1,  50,   5,  12,   8,   2,   1,  37,  27,   9,  17,
         11,   5,  29,  10,  99,   4,   1,  13, 100,   6, 101,   2, 102,
        103, 104,   3,  29,   1,  52,  50,  18,   8,  27,   9,  17,   6,
         29, 105,   1,  28,  48,   5,  12, 106, 107,   2,   3]),
 32)

In [15]:
y = tf.keras.utils.to_categorical(y, num_classes=VOCAB_SIZE)

In [16]:
X.shape, y.shape

((732, 50), (732, 310))

In [17]:
tf_model = tf.keras.models.Sequential(layers=[
    tf.keras.layers.Embedding(input_dim=VOCAB_SIZE, output_dim=SEQUENCE_LEN, input_length=SEQUENCE_LEN),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(100, return_sequences=True)),
    tf.keras.layers.LSTM(100, return_sequences=True),
    tf.keras.layers.LSTM(100),
    tf.keras.layers.Dense(100, activation="relu"),
    tf.keras.layers.Dense(VOCAB_SIZE, activation="softmax")
])

Metal device set to: Apple M1

systemMemory: 8.00 GB
maxCacheSize: 2.67 GB



2021-11-30 22:05:16.467312: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2021-11-30 22:05:16.467402: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [18]:
tf_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 50, 50)            15500     
_________________________________________________________________
bidirectional (Bidirectional (None, 50, 200)           120800    
_________________________________________________________________
lstm_1 (LSTM)                (None, 50, 100)           120400    
_________________________________________________________________
lstm_2 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense (Dense)                (None, 100)               10100     
_________________________________________________________________
dense_1 (Dense)              (None, 310)               31310     
Total params: 378,510
Trainable params: 378,510
Non-trainable params: 0
__________________________________________________

In [19]:
tf_model.compile(loss=["categorical_crossentropy"], optimizer="adam", metrics=["accuracy"])

In [20]:
tf_model.fit(X, y, epochs=100, batch_size=256)

2021-11-30 22:05:17.072992: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2021-11-30 22:05:17.073165: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


Epoch 1/100


2021-11-30 22:05:18.217798: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2021-11-30 22:05:18.408138: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2021-11-30 22:05:18.416926: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2021-11-30 22:05:18.541714: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2021-11-30 22:05:18.617022: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2021-11-30 22:05:18.749843: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2021-11-30 22:05:18.898505: I tensorflow/core/grappler/optimizers/cust

3/3 [==============================] - 3s 388ms/step - loss: 5.7359 - accuracy: 0.0328
Epoch 2/100
3/3 [==============================] - 0s 155ms/step - loss: 5.7313 - accuracy: 0.0574
Epoch 3/100
3/3 [==============================] - 0s 156ms/step - loss: 5.7239 - accuracy: 0.0574
Epoch 4/100
3/3 [==============================] - 1s 172ms/step - loss: 5.7037 - accuracy: 0.0574
Epoch 5/100
3/3 [==============================] - 0s 156ms/step - loss: 5.6229 - accuracy: 0.0574
Epoch 6/100
3/3 [==============================] - 0s 155ms/step - loss: 5.3895 - accuracy: 0.0574
Epoch 7/100
3/3 [==============================] - 1s 176ms/step - loss: 5.2621 - accuracy: 0.0574
Epoch 8/100
3/3 [==============================] - 0s 163ms/step - loss: 5.1931 - accuracy: 0.0574
Epoch 9/100
3/3 [==============================] - 1s 183ms/step - loss: 5.1390 - accuracy: 0.0574
Epoch 10/100
3/3 [==============================] - 1s 163ms/step - loss: 5.0933 - accuracy: 0.0574
Epoch 11/100
3/3 [===

In [21]:
def generate_text_sequence (tfmodel, tokenizer, text_seq_length, raw_text, n_words):
    texts = list()
    for index in range (n_words):
        enconded = tokenizer.texts_to_sequences(raw_text)[0]
        encoded_padded = tf.keras.preprocessing.sequence.pad_sequences(np.array([enconded]), maxlen=text_seq_length,  truncating="post", padding="post")
        y_prdicted = tfmodel.predict_classes(encoded_padded)

        predicted_word = ""
        for word, index in tokenizer.word_index.items():
            if index == y_prdicted:
                predicted_word = word
                break

        raw_text = raw_text + predicted_word
        texts.append(predicted_word)

    return " ".join(texts)

In [26]:
tf.cast(tf_model.predict(X[-50:]), dtype=np.int32)

<tf.Tensor: shape=(50, 310), dtype=int32, numpy=
array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int32)>